# **Programming for Biologists 3: Unlocking Insights from A FASTA File**

**Welcome, data explorer!**

In biological research, we often encounter sequences (DNA, RNA, proteins) stored in a common format called **FASTA**. A FASTA file is essentially a text file where each sequence has a header line (starting with `>`) followed by the actual sequence data.

Imagine you've received a FASTA file containing hundreds or thousands of newly sequenced genes or protein variants. You can't manually scroll through all of them! This notebook will show you how to use Python to automatically read these files, extract the valuable information, and perform basic analyses to gain quick insights into your sequences.

This is a crucial skill for anyone working with genomic or proteomic data, allowing you to quickly understand the characteristics of your sequence collections.

---

## **1. Setting Up: Creating a Sample FASTA File**

To make this notebook immediately runnable, we'll first create a small, artificial FASTA file directly in Python. In your real work, you would simply replace `"sample_genes.fasta"` with the name of your actual FASTA file.

Each entry in a FASTA file has:
* A **header line** starting with `>` (e.g., `>gene_A_human | Homo sapiens gene A`). This usually contains the sequence ID and description.
* One or more lines of the **sequence data** itself (e.g., `ATGCGT...`).

In [ ]:
# Define the content for our sample FASTA file
fasta_content = """
>gene_1 | Hypothetical protein from bacteria
ATGCGTACGTAGCTAGCTAGCTAGCTACGATGCATGCA
GCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTACGTACG
>gene_2 | Ribosomal RNA from yeast
AUGCAGUCAGUCAGUCAUGCAUGCUGA
>gene_3 | Viral capsid protein
ATGGCAAGGTTTCAGGTAACCAAA
GGGTTTCCCG
>gene_4 | Short regulatory element
ATGC
>gene_5 | Longest human gene fragment
GATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATC
GATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATC
GATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATC
>gene_6 | Another bacterial gene
ATGCGCGCGCGCGCGCGCGCGCGCGCGCGCGCGCGCGCGCGCGCGCGCGCGCGCGCGCGC
GCGCGCGCGCGCGCGCGCGCGCGCGCGCGCGCGCGCGCGCGCGCGCGCGCGCGCGCGCGC
ATATATATATATATATATATATATATATATATATATATATATATATATATATATATATAT
ATATATATATATATATATATATATATATATATATATATATATATATATATATATATATAT
"""

# Name of our sample file
file_name = "sample_genes.fasta"

# Write the content to the file
with open(file_name, "w") as f:
    f.write(fasta_content)

print(f"Successfully created '{file_name}' with sample data.")

---

## **2. Parsing the FASTA File: Extracting IDs and Sequences**

Now that we have our FASTA file, let's write Python code to read it and pull out the important pieces: the sequence ID (from the header) and the sequence itself.

We'll store these in a **dictionary**, where each sequence ID will be a `key` and its corresponding sequence will be the `value`.

In [ ]:
sequences_dict = {} # This dictionary will store our parsed data
current_sequence_id = None # To keep track of the current sequence's ID

print(f"--- Parsing '{file_name}' ---")

with open(file_name, "r") as fasta_file:
    for line in fasta_file:
        line = line.strip() # Remove leading/trailing whitespace (like newlines)

        if not line: # Skip empty lines
            continue

        if line.startswith('>'):
            # This is a header line
            # Extract the ID (everything after '>')
            current_sequence_id = line[1:]
            # Initialize an empty string for the sequence associated with this ID
            sequences_dict[current_sequence_id] = ""
        else:
            # This is a sequence line
            if current_sequence_id is not None:
                # Add (append) this line of sequence to the current ID's sequence
                # Convert to uppercase to make analysis consistent (e.g., 'a' vs 'A')
                sequences_dict[current_sequence_id] += line.upper()
            else:
                print("Warning: Sequence line found before any header. Skipping:", line)

print("--- Parsing complete! ---")
print(f"Found {len(sequences_dict)} sequences.")

# Let's print the first few entries to see what we've got
print("\nFirst 3 parsed sequences (ID and start of sequence):")
count = 0
for seq_id, sequence in sequences_dict.items():
    if count < 3:
        print(f"ID: {seq_id[:30]}... | Sequence: {sequence[:40]}...")
        count += 1
    else:
        break

**Why this matters :** This is the foundational step for almost any sequence analysis. Once parsed, your sequences are in a structured format (a dictionary) that Python can easily work with.

---

## **3. Gaining Insights: Basic Sequence Analysis**

With our sequences parsed and stored in `sequences_dict`, we can now start asking questions and extracting useful information.

### **3.1 Sequence Lengths**

The length of a sequence can tell us a lot (e.g., coding vs. non-coding RNA, full-length protein vs. fragment). We can easily find the length of each sequence using `len()`.

In [ ]:
sequence_lengths = {} # To store ID -> Length mapping
all_lengths = [] # To store all lengths for overall statistics

print("--- Sequence Lengths ---")
for seq_id, sequence in sequences_dict.items():
    length = len(sequence)
    sequence_lengths[seq_id] = length
    all_lengths.append(length)
    print(f"ID: {seq_id[:30]}... | Length: {length} bases")

print(f"\nAverage sequence length: {sum(all_lengths) / len(all_lengths):.2f}")
print(f"Longest sequence length: {max(all_lengths)}")
print(f"Shortest sequence length: {min(all_lengths)}")

### **3.2 GC Content**

GC content (percentage of Guanine and Cytosine bases) is a fundamental property of DNA that can vary significantly between organisms and even within different regions of a genome. It's often related to melting temperature and gene stability.

Recall our `calculate_gc_content` function from a previous notebook (or let's define it again for this one):

In [ ]:
def calculate_gc_content(dna_sequence):
    """Calculates the percentage of G and C bases in a DNA sequence."""
    # Ensure sequence is uppercase for consistent counting
    dna_sequence = dna_sequence.upper()
    g_count = dna_sequence.count('G')
    c_count = dna_sequence.count('C')
    total_bases = len(dna_sequence)

    if total_bases == 0:
        return 0.0
    else:
        gc_percent = ((g_count + c_count) / total_bases) * 100
        return gc_percent

print("--- GC Content for Each Sequence ---")
gc_contents = {} # To store ID -> GC Content mapping
for seq_id, sequence in sequences_dict.items():
    gc = calculate_gc_content(sequence)
    gc_contents[seq_id] = gc
    print(f"ID: {seq_id[:30]}... | Length: {len(sequence)} | GC Content: {gc:.2f}%")

# Find the sequence with highest/lowest GC content (if you want to be fancy)
if gc_contents:
    highest_gc_id = max(gc_contents, key=gc_contents.get)
    lowest_gc_id = min(gc_contents, key=gc_contents.get)
    print(f"\nSequence with highest GC content ({gc_contents[highest_gc_id]:.2f}%): {highest_gc_id}")
    print(f"Sequence with lowest GC content ({gc_contents[lowest_gc_id]:.2f}%): {lowest_gc_id}")

### **3.3 Finding Specific Motifs/Patterns**

You might be interested if certain short sequences (motifs) are present in your longer sequences. For example, a restriction enzyme recognition site (`GAATTC` for EcoRI), or a protein domain signature.

In [ ]:
motif = "ATGCAG" # Let's search for this motif
found_in_sequences = []

print(f"--- Searching for motif '{motif}' ---")
for seq_id, sequence in sequences_dict.items():
    # Check if the motif exists in the sequence (case-insensitive search is good practice)
    if motif.upper() in sequence.upper():
        found_in_sequences.append(seq_id)
        print(f"Motif found in: {seq_id}")

if found_in_sequences:
    print(f"\nTotal sequences containing '{motif}': {len(found_in_sequences)}")
else:
    print(f"Motif '{motif}' not found in any sequence.")

**Your Turn! (Exercise 1)**

1.  **Count Specific Bases:** Modify the code to count the total number of 'A' bases across *all* sequences in your `sequences_dict`.
    *Hint: You'll need a loop and `sequence.count('A')`*.
2.  **Find Short Sequences:** Create a list of all sequence IDs that have a length less than 50 bases. Print this list.
3.  **Search for another motif:** Pick a new 4-base DNA motif (e.g., `"TATA"` or `"GATA"`) and find out which sequences contain it. Print the IDs of the sequences where it's found.

In [ ]:
# Write your code for Exercise 1 here!

# 1. Count Total 'A' Bases
total_a_bases = 0
for seq_id, sequence in sequences_dict.items():
    total_a_bases += sequence.count('A')
print(f"\nTotal 'A' bases across all sequences: {total_a_bases}")

# 2. Find Short Sequences
short_sequence_ids = []
for seq_id, sequence in sequences_dict.items():
    if len(sequence) < 50:
        short_sequence_ids.append(seq_id)
print("\nShort sequences (length < 50):", short_sequence_ids)

# 3. Search for a new motif
new_motif = "GATA"
sequences_with_new_motif = []
print(f"\n--- Searching for new motif '{new_motif}' ---")
for seq_id, sequence in sequences_dict.items():
    if new_motif.upper() in sequence.upper():
        sequences_with_new_motif.append(seq_id)
        print(f"Motif '{new_motif}' found in: {seq_id}")

if sequences_with_new_motif:
    print(f"Total sequences containing '{new_motif}': {len(sequences_with_new_motif)}")
else:
    print(f"Motif '{new_motif}' not found in any sequence.")

---

## **4. Advanced Insight (Optional): Visualizing Length Distribution**

For larger datasets, looking at a list of lengths isn't very helpful. A histogram (a type of bar chart) can quickly show you the distribution of sequence lengths. Do you have many short sequences? A few very long ones? Most are medium?

To create plots, we often use a powerful library called `matplotlib.pyplot` (we usually import it as `plt`).

In [ ]:
# Make sure you have matplotlib installed: !pip install matplotlib

import matplotlib.pyplot as plt

if all_lengths: # Only try to plot if we actually have lengths
    plt.figure(figsize=(10, 6))
    plt.hist(all_lengths, bins=10, edgecolor='black') # 'bins' divides data into groups
    plt.title('Distribution of Sequence Lengths')
    plt.xlabel('Sequence Length (bases)')
    plt.ylabel('Number of Sequences')
    plt.grid(axis='y', alpha=0.75)
    plt.show()
else:
    print("No sequence lengths to plot. Ensure your FASTA file was parsed correctly.")

**Why this matters:** Visualizations are incredibly important for quickly understanding trends and quality control in large biological datasets. A quick look at a length distribution can tell you if your sequencing run produced fragments of expected sizes or if there are issues.

---

## **Conclusion: Your Gateway to Sequence Analysis**

You've successfully built a Python workflow to:

1.  **Prepare** a sample FASTA file.
2.  **Parse** FASTA data, extracting sequence IDs and sequences.
3.  **Analyze** key properties like sequence lengths and GC content.
4.  **Search** for specific motifs.
5.  (Optionally) **Visualize** data distributions.

These are fundamental operations that form the basis of more complex bioinformatics tasks like genome assembly, gene annotation, phylogenetic analysis, and functional prediction. With these skills, you're well-equipped to start exploring your own biological sequence data!